# TensorCRO tutorial: Max Ones.

The CRO-SL algorithm is a general-purpose algorithm for finding the maximum of a function. It is a stochastic algorithm, which means that it is not guaranteed to find the global maximum, but it is guaranteed to find a local maximum. The algorithm is based on the idea of the coral reefs reproduction and is guaranteed to find a local maximum if the function is convex, and it is likely to find the global maximum if the selected operators are good enough.

The algorithm is implemented in the TensorCRO library, which is a Python library for the CRO-SL algorithm.

To implement a CRO-SL algorithm with TensorCRO library you must follow the following steps:

1. [Understanding CRO-SL.](#understanding)
2. [Define a fitness function to be MAXIMIZED.](#fitness)
3. [Define the CRO-SL algorithm parameters.](#parameters)
4. [Define the CRO-SL substrates.](#substrates)
5. [Run the CRO-SL algorithm.](#fit)
6. [Watch the replay.](#replay)
7. [Building new substrates.](#new)
8. [Connecting to SlackBot.](#slackbot)

## Understanding CRO-SL. <a name="understanding"></a>

CRO-SL is a heuristic and bio-inspired algorithm that simulates the reproduction of coral reefs for parameter optimization. Figure shows the steps that the algorithm follows to find the maximum of a function.

The problem solved can be defined a follows:


```math
Given a function f(x, y, z, ...) to be maximized, find the maximum value of f(x) and the corresponding x, y, z, ... parameters that maximize the function f.
```

In this tutorial, we will maximize 5 parameters that their sum must be maximized. The best result will be the parameters with the highest value possible. This is known as the Max Ones problem.

The algorithm follows 5 steps:

1. **Initialize the population**: The population is initialized with random values in the range of each parameter. Some of the initialized values may correspond to alive corals and others to dead corals. The portion of alive corals in the reef is defined by the ``rho`` parameter.
2. **Broadcast spawning**: ``Fb`` portion of alive corals are selected to use crossover operators (known as ``substrates``). The selected corals are used to create new corals, known as larvae. Each substrate has its own parameters, and it must be defined previously by the user.
3. **Select the best individuals**: ``1 - Fb`` portion of the alive corals are selected to experiment a Gaussian Mutation, with 1% of variance for each parameter. New larvae are created from the mutated corals.
4. **Larvae setting**: The current larvae are set in the population. The larvae fight for random positions of the coral, if their fitness is better than the random position, the larvae replaces the random position. If the larvae is worse than the random position, the larvae is discarded. This step is repeated ``k`` times, and if the larvae is not able to replace any position, it is definitely discarded. If a larvae fights with a dead coral, the larvae replaces the dead coral.
5. **Fragmentation**: ``Fa`` portion of corals are selected to be fragmented. The selected corals are replaced by new corals with random values in the range of each parameter, following a ``Larvae Setting`` process.
6. **Depredation**: ``Fd`` portion of the worst corals are selected to be depredated. The selected corals can die with a probability of ``Pd``.

Overall, the algorithm has 6 parameters and a list of substrates. All parameters are real values between 0 and 1, except ``k`` which is a natural number with no upper limit.

<p align="center">
    <img src="../cro_process.png" width="400px">
</p>


# Define the fitness function. <a name="fitness"></a>

In this tutorial, we want to maximize the sum of all the parameters for each coral, so we define the fitness function as follows:

In [1]:
def non_optimized_fitness_function(x: list[list[float]]) -> list[float]:
    return [sum(individual) for individual in x]

As this framework makes use of TensorFlow, if your fitness function makes use of TensorFlow operations it will run even faster. The input of the fitness function is a list of larvae with NxM dimensions, where N is the number of larvae and M is the number of parameters. The output of the fitness function is a list of fitness values with N dimensions. The fitness function must be defined as follows in TensorFlow:

In [2]:
import tensorflow as tf
@tf.function
def fitness_function(larvae: tf.Tensor) -> tf.Tensor:
    return tf.reduce_sum(larvae, axis=-1)

2023-10-24 18:17:16.330577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 18:17:17.265207: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-24 18:17:17.265279: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-24 18:17:17.265285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In case you want to minimize the function, you can add a minus sign to the fitness function in the return value.

# Define the CRO-SL algorithm parameters. <a name="parameters"></a>

The first thing to do is define the parameters of the individuals, this means, the bounds of each of the parameters. In this example, we will use 5 parameters, with different bounds. They must be defined as a 2 row tensor, where the first row is the lower bound and the second row is the upper bound. The lower bound must be smaller than the upper bound. They must be converted to a Tensor.

The second thing is to define the reef shape. It is a rectangular shape where the columns MUST BE DIVISIBLE BY THE NUMBER OF SUBSTRATES. For example, we will use 3 substrates, so the reef columns must be divisible by 3. The reef rows are not limited.

The third thing to do is define the parameters, the algorithm already has implemented a recommended set of parameters, but you can define your own parameters.

In [3]:
from tensorcro import TensorCro, HarmonySearch, RandomSearch, MultipointCrossover

# Define parameters:
bounds = tf.convert_to_tensor([[-1, -0.5, 0, 0, 0], [0, 0, 1, 0.5, 1]], dtype=tf.float32)  # Upper and lower limits.
# The reef shape (81 possible individuals):
reef_shape = (9, 9)  # Reef shape, in this case, squared. 3 columns per substrate.
# A custom CRO parameter:
k = 2  # Number of larvae setting trials.

2023-10-24 18:17:18.319307: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-24 18:17:18.442455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-24 18:17:18.442592: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-24 18:17:18.443945: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

# Define the CRO-SL substrates. <a name="substrates"></a>

The last thing to do is define the substrates. The substrates are the operators that will be used to create new larvae. The framework already has implemented a set of substrates, but you can define your own substrates.

To define a substrate, you can look for your favourite and instance its class. The algorithms end up with ``Search`` while ``Crossover`` are general purpose crossover operators. Algorithms that make calls to the fitness function are not supported yet.

In [4]:
# Substrate definition.
harmony_search_substrate = HarmonySearch(bounds)  # Harmony Search algorithm used as a crossover method.
random_search_substrate = RandomSearch(bounds)  # Random Search algorithm used as a crossover method.
multipoint_crossover_substrate = MultipointCrossover([3])  # Multipoint Crossover algorithm used as a crossover method.
substrates = [harmony_search_substrate, random_search_substrate, multipoint_crossover_substrate]  # List of substrates.
# CRO instance.
cro = TensorCro(reef_shape=reef_shape, subs=substrates, k=k)  # Initialize the CRO-SL algorithm.

# Run the CRO-SL algorithm. <a name="fit"></a>

Now, we can run the algorithm. The algorithm will run until the maximum number of iterations is reached. The algorithm will return the current reef, sorted by fitness.

To do so, you must call the fitness method. The fitness method has 3 main parameters, the fitness function, the maximum number of iterations and the parameter bounds.

The fit method also has other 3 parameters. ``device`` tells the device that will run the fit method. In this case, we are using the GPU. To use a GPU you must have CUDA installed and a TensorFlow-compatible GPU. ``seed`` is the seed that will be used to initialize the random number generator, it is implemented to repeat the fit process in the same way, useful for research experiments. ``shards`` is an integer telling the algorithm to save the results each `shards` iterations. For example, if we are running 1000 iterations and shards are set to 100, each 100 iterations we will save the results. This is useful to run the algorithm in a cluster and save the results in case of a crash, and also for watching the evolution replay afterward.

**You can also create your own initialization reef, so you can continue a fit process from a previous saved point!** Just specify a tuple with the reef and the fitness of the reef int the `init` parameter.

In [8]:
# Initial reef, if provided:
initial_reef = tf.zeros((9, 9, 5), dtype=tf.float32)
initial_fitness = fitness_function(initial_reef)
# Fit method.
cro.fit(fitness_function, bounds, max_iter=50, device='/CPU:0', seed=0, shards=5, init=(initial_reef, initial_fitness), tf_compile=True)  # Run the algorithm.

5/5 [==============================] - 4s 29ms/step - Best fitness: 2.4389


(<tf.Tensor: shape=(81, 5), dtype=float32, numpy=
 array([[ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0.5       ,  1.        ],
        [ 0.        ,  0.        ,  1.        ,  0

Specify `tf_compile = False` when your fitness function cannot be implemented in GPU but you still want to use this framework.

# Watch the evolution replay. <a name="replay"></a>

Once we selected an optimal number of iterations, we can watch the evolution replay. The evolution replay is watched though a Tkinter GUI. The parameters to launch the GUI are:

- ``path: str``: Path of the replay.
- ``mp: bool``: Launch the GUI in a different process.
- ``lock: bool``: Waits the GUI to be closed to continue the program.

For that we can save the last fit launched in the path we want with the `save_replay` method, but we are not going to use it in this tutorial. If you want to watch the last replay, you can omit the `path` parameter.

In [9]:
cro.watch_replay()

<Process name='Process-1' pid=5235 parent=4652 started>

# Building new substrates. <a name="substrates"></a>

The framework already has implemented a set of substrates, but you can define your own substrates. To define a substrate, you can implement your substrate from zero or use the ``ComposedSubstrate`` class to define a sequential set of substrates. For example, we implement the genetic algorithm as a sequential of `MultiPointCrossover` and `Mutation` substrates.

In [10]:
from tensorcro import ComposedSubstrate, Mutation

genetic_algorithm = ComposedSubstrate(MultipointCrossover([3]), Mutation('gaussian', stddev=0.1), name='GeneticAlgorithm')

We can also implement the genetic algorithm as a new substrate using the class ``Substrate``. You must implement a method called `_call` that takes a list of larvae to be crossed and returns a list of larvae crossed. The ``Substrate`` class has a method called `__call__` that calls the `_call` method and returns the crossed larvae. 

Recall that directives is a parameter with the minimum and maximum value specifications for each parameter, of shape (2, ndims). **All algorithms are responsible for dealing with their solutions in the specified range. TensorCRO will autoclip the values after the new population is given. This means that the user must deal with overflow and domain issues.**
*DON'T FORGET KWARGS IN _CALL METHOD*

In [11]:
from tensorcro.substrates import CROSubstrate

class GeneticAlgorithm(CROSubstrate):
    def __init__(self, directives, crossover_points: list[int], mutation_stddev: float):
        self.directives = directives
        self.crossover_points = crossover_points
        self.mutation_stddev = mutation_stddev

    def _call(self, individuals: tf.Tensor, **kwargs) -> tf.Tensor:
        # Use your TensorFlow implementation, I will use the pre-defined classes.
        return Mutation('gaussian', stddev=self.mutation_stddev)(MultipointCrossover(self.crossover_points)(individuals))

In 2.0, we can now create a substrate for stateful optimization algorithms. This is so for `PSO` and `SimulatedAnnealing` algorithms that must store registers of speed, local best or fitness values. 

To do so, we must implement the `CROSubstrate` as we did before, but we must add the shape attribute. 

First, we have to precompute the size of our substrate in this specific problem. Imagine we are implementing `SimulatedAnnealing` algorithm with a random search in a 10x20 reef. The first substrate will have a 10x10 grid, while the other will have the same size.

The substrate reef is computed as follows:
```python
substrate_reef = (number_of_rows, number_of_columns // number_of_substrates)
```

**IMPORTANT:**
All attributes used in `_call()` must be a `tf.Variable`. This means that we must specify the initial value in the constructor, and it is the reason of pre-computing the substrate shape in prior.

In [12]:
reef_shape = (10, 20)
substrate_shape = (10, 10)

In this example, I will implement `MyStatefuleAlgorithm` that will increase by x% the input values in population. x% starts with 1%, but it is increased by 1% every time a coral is alive on the position. The algorithm is implemented as follows:

1. Get the positions of the given individuals.
2. Get the current increment values of the current positions.
3. Multiply the individuals by the increments.
4. Update the increments by 1%, only in the used positions.
5. Return the multiplied individuals.

In [13]:
from tensorcro.substrates import CROSubstrate

class MyStatefulAlgorithm(CROSubstrate):
    def __init__(self, directives: tf.Tensor, shape: tuple):
        """
        This algorithm increments by +1% the individuals given each iteration.
        :param directives: The minimum and maximum for each parameter.
        :param shape: The shape of the substrate reef.
        """
        self.directives = tf.constant(directives, dtype=tf.float32)
        initial_value = tf.ones(shape) * 1.01
        self.increment = tf.Variable(initial_value=initial_value, dtype=tf.float32)

    def _call(self, individuals: tf.Tensor, **kwargs) -> tf.Tensor:
        # Get alive corals in the substrate:
        coral_idx = kwargs['ids']
        # Get the reef:
        increments = tf.gather_nd(self.increment, coral_idx)
        # Increment the individuals:
        incremented = individuals * increments
        # Update the increment variable by 1:
        new_increments = tf.tensor_scatter_nd_update(self.increment, coral_idx, increments + 0.01)
        self.increment.assign(new_increments)
        # Return the incremented individuals:
        return incremented
    
algorithm = MyStatefulAlgorithm(directives=bounds, shape=substrate_shape)

Note that `**kwargs` have 2 attributes in case the user needs them:

- `ids`: The positions of the individuals in the reef.
- `fitness`: The fitness of the individuals.

## Connecting to SlackBot. <a name="slackbot"></a>

You can connect the algorithm to a SlackBot to watch the evolution of the algorithm in real-time. To do so, you must create a SlackBot and get the token. You can follow the instructions here to create a SlackBot: https://api.slack.com/bot-users. After creating a bot and inviting him to your channel, you will have a `BOT-TOKEN`.

In this case we will use `TensorCRO-Bot`. To use it, you need to import the `SlackCallback` class from the package. Then, you must create a `SlackCallback` instance with the token of your SlackBot. Finally, you must pass the callback to the `fit` method. You can also use `exception_handler()` and `end()` methods to send messages to SlackBot.

* `exception_handler()` will send a message with the exception raised by the code, a fitness plot and a backup of the last seen population.
* `end()` will send a message with the solution of the algorithm, a fitness plot and a backup of the last seen population.

After that code, you will receive 5 type of messages:

1. On run:
    -  A message with the simulation status.
    -  A fitness plot.
2. On exception:
    -  A message with the exception raised by the code.
    -  A fitness plot.
    -  A backup of the last seen population as a np.array.
3. On finish:
    -  A message with the finish status.
    -  A fitness plot.
    -  The solution of the algorithm as a np.array.

In [1]:
from tensorcro import SlackCallback
# Set up the callback:
slack_callback = SlackCallback(bot_token='<YOUR-BOT-TOKEN>', channel='<YOUR-CHANNEL>', username='<YOUR-SLACK-DISPLAY-NAME>', simulation_name='<A-NAME-FOR-YOUR-SIMULATION>')
# Run the algorithm:
try:
    best = cro.fit(fitness_function, bounds, max_iter=50, device='/CPU:0', seed=0, shards=5, callback=slack_callback)
    slack_callback.end(best[0].numpy())
except Exception as e:
    slack_callback.exception_handler(e)

2023-10-25 18:19:50.124657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-25 18:19:50.661843: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-25 18:19:50.661886: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-25 18:19:50.661892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

TypeError: unsupported operand type(s) for /: 'int' and 'NoneType'

All done! Good job!